# Nearest neighbor for handwritten digit recognition

In this notebook we will build a classifier that takes an image of a handwritten digit and outputs a label 0-9.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt 
import time
# import os
# os.getcwd()
# os.listdir()

### MNIST dataset

`MNIST` is a classic dataset in machine learning, consisting of 28x28 gray-scale images handwritten digits. The original training set contains 60,000 examples and the test set contains 10,000 examples. In this notebook we will be working with a subset of this data: a training set of 7,500 examples and a test set of 1,000 examples.

In [ ]:
path = !find ../../_data | grep -i mnist/
path

In [ ]:
## Load the training set
train_data = np.load('../../_data/MNIST/train_data.npy')
train_labels = np.load('../../_data/MNIST/train_labels.npy')

## Load the testing set
test_data = np.load('../../_data/MNIST/test_data.npy')
test_labels = np.load('../../_data/MNIST/test_labels.npy')

#### Sanity check dimensions

In [ ]:
## Print out their dimensions
print("Training dataset dimensions: ", np.shape(train_data))
print("Number of training labels: ", len(train_labels))
print("Testing dataset dimensions: ", np.shape(test_data))
print("Number of testing labels: ", len(test_labels))

#### Sanity check label distribution

In [ ]:
train_digits, train_counts = np.unique(train_labels, return_counts=True)
print("Training set distribution:")
print(dict(zip(train_digits, train_counts)))

test_digits, test_counts = np.unique(test_labels, return_counts=True)
print("Test set distribution:")
print(dict(zip(test_digits, test_counts)))

### Visualizing the data
Each data point is stored as 784-dimensional vector. To visualize a data point, we first reshape it to a 28x28 image.

In [ ]:
## Define a function that displays a digit given its vector representation
def show_digit(x, **kwargs):
    ax = kwargs.get('ax', plt.gca())
    ax.axis('off')
    ax.imshow(x.reshape((28,28)), cmap=plt.cm.gray)
    return

## Define a function that takes an index into a particular data set ("train" or "test") and displays that image.
def vis_image(index, dataset="train"):
    if(dataset=="train"): 
        show_digit(train_data[index,])
        label = train_labels[index]
    else:
        show_digit(test_data[index,])
        label = test_labels[index]
    print("Label " + str(label))
    return

## View the first data point in the training set
vis_image(0, "train")
plt.show()

## Now view the first data point in the test set
vis_image(0, "test")

### KNN distance metrics

To compute nearest neighbors in our data set, we need to first be able to compute distances between data points. 
A natural distance function is _Euclidean distance aka the L2-norm_: for two vectors $x, y \in \mathbb{R}^d$, their Euclidean distance is defined as   
$$L2-norm = \|x - y\|^2 = \sqrt{\sum_{i=1}^d (x_i - y_i)^2}.$$

Often we omit the square root and simply compute _squared Euclidean distance_:
$$\|x - y\|^2 = \sum_{i=1}^d (x_i - y_i)^2.$$

Another distance metric is the _Manhattan distance or taxicab distance aka the L1-norm_:   
$$L1-norm = \|x - y\| = \sum_{i=1}^d |x_i - y_i|.$$

In [ ]:
def L1(x, y):
    """L1 (Manhattan) distance.
    :param x, y: vectors x, y
    :returns: distance"""
    return np.sum(np.abs(x-y))

In [ ]:
def L2(x, y):
    """L2 (Euclidean) distance.
    :param x, y: vectors x, y
    :returns: distance"""
    return np.sum(np.square(x-y))

In [ ]:
def L_norm(x, y, norm):
    """L1 (Manhattan) distance.
    :param x, y: vectors x, y
    :returns: distance"""
    return norm(x, y)

In [ ]:
## Compute distance between a seven and a one in our training set.
print("Distance from 7 to 1: ", L_norm(train_data[4,], train_data[5,], L2))
print("Distance from 7 to 1: ", L_norm(train_data[4,], train_data[5,], L1))

## Compute distance between a seven and a two in our training set.
print("Distance from 7 to 2: ", L_norm(train_data[4,], train_data[1,], L2))
print("Distance from 7 to 2: ", L_norm(train_data[4,], train_data[1,], L1))

## Compute distance between two seven's in our training set.
print("Distance from 7 to 7: ", L_norm(train_data[4,], train_data[7,], L2))
print("Distance from 7 to 7: ", L_norm(train_data[4,], train_data[7,], L1))

### Computing nearest neighbors

Now that we have a distance function defined, we can now turn to nearest neighbor classification. 

In [ ]:
## Takes a vector x and returns the index of its nearest neighbor in train_data
def find_NN(x, norm):
    # Compute distances from x to every row in train_data
    distances = [L_norm(x, train_data[i,], norm) for i in range(len(train_labels))]
    # Get the index of the smallest distance (in train)
    return np.argmin(distances)

## Takes a vector x and returns the class of its nearest neighbor in train_data
def NN_classifier(x, norm):
    # Get the index of the the nearest neighbor
    index = find_NN(x, norm)
    # Return its class
    return train_labels[index]

In [ ]:
def NN_test(index, norm):
    """
    :param: index of test_data
    :returns: None
    :plot: images of test and predicted labels"""
    true_title = '''
    Test label: {}'''.format(
        test_labels[index])
    
    pred_title = '''
    NN Train label: {}
    Correct class: {}'''.format(
        NN_classifier(test_data[index, ], norm), 
        test_labels[index]==NN_classifier(test_data[index, ], norm))
    
#     print(title)
    fig, axes = plt.subplots(1, 2)
    axes[0].set_title(true_title)
    axes[1].set_title(pred_title)
    show_digit(test_data[index,], ax=axes[0])
    show_digit(train_data[find_NN(test_data[index, ], norm), ], ax=axes[1])
    

In [ ]:
NN_test(39, L1)
NN_test(39, L2)
NN_test(100, L1)
NN_test(100, L2)

In [ ]:
for i in np.random.permutation(range(test_data.shape[0]))[:10]:
    NN_test(i)

### Classify full test set

Note that to classify __each test point__, our code takes a __full pass over each of the 7500 training examples__. Thus we should not expect testing to be very fast. The following code takes about 50-60 seconds on MacBookPro 2016 (2,9 GHz Intel Core i5, 8 GB 2133 MHz LPDDR3).

In [ ]:
t_before = time.time()
test_predictions = [NN_classifier(test_data[i, ]) for i in range(len(test_labels))]
t_after = time.time()

In [ ]:
## Compute the error
err_positions = np.not_equal(test_predictions, test_labels)
error = float(np.sum(err_positions))/len(test_labels)

print("Error of nearest neighbor classifier: ", error)
print("Classification time (seconds): ", t_after - t_before)

## Faster NN algorithms

Performing nearest neighbor classification in the way we have presented requires a full pass through the training set in order to classify a single point. If there are $N$ training points in $\mathbb{R}^d$, this takes $O(N d)$ time.

Fortunately, there are faster methods to perform nearest neighbor look up if we are willing to spend some time preprocessing the training set. __`scikit-learn`__ has __fast implementations of two useful nearest neighbor data structures: the _ball tree_ and the _k-d tree___. 

### Ball tree

A tree implementation of the training set. Traversing through a tree is computationaly more efficient.
First the training set must be converted to a tree, this initial cost is fractional and will be paid back in every search.

In [ ]:
from sklearn.neighbors import BallTree

In [ ]:
## Build nearest neighbor structure on training data
t_before = time.time()
ball_tree = BallTree(train_data)
t_after = time.time()

In [ ]:
## Compute training time
t_training = t_after - t_before
print("Time to build data structure (seconds): ", t_training)

In [ ]:
## Get nearest neighbor predictions on testing data
t_before = time.time()
test_neighbors = np.squeeze(ball_tree.query(test_data, k=1, return_distance=False))
ball_tree_predictions = train_labels[test_neighbors]
t_after = time.time()

In [ ]:
## Compute testing time
t_testing = t_after - t_before
print("Time to classify test set (seconds): ", t_testing)

In [ ]:
## Verify that the predictions are the same
print("Ball tree produces same predictions as above? ", np.array_equal(test_predictions, ball_tree_predictions))

### KDTree

In [ ]:
from sklearn.neighbors import KDTree

In [ ]:
## Build nearest neighbor structure on training data
t_before = time.time()
kd_tree = KDTree(train_data)
t_after = time.time()

In [ ]:
## Compute training time
t_training = t_after - t_before
print("Time to build data structure (seconds): ", t_training)

In [ ]:
## Get nearest neighbor predictions on testing data
t_before = time.time()
test_neighbors = np.squeeze(kd_tree.query(test_data, k=1, return_distance=False))
kd_tree_predictions = train_labels[test_neighbors]
t_after = time.time()

In [ ]:
## Compute testing time
t_testing = t_after - t_before
print("Time to classify test set (seconds): ", t_testing)

In [ ]:
## Verify that the predictions are the same
print("KD tree produces same predictions as above? ", np.array_equal(test_predictions, kd_tree_predictions))